# PROJECT. Оценка эффективности новой фичи

### Для понимания эффективности новой фичи нужно:
### 1. Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до и после внедрения фичи.
### 2. Сформировать выводы по анализу.


In [47]:
# Загружаем необходимые библиотеки
! pip install psycopg2
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

In [48]:
# Получаем данные из базы по количеству зарегистрированных и активных пользователей,
# приводим дату в нужный формат
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users_count = pd.DataFrame(getRegisteredUsersCount())
registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])

def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())
for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts[col] = pd.to_datetime(active_users_count_with_cohorts[col])

In [49]:
# Соединяем две таблицы
retention_table = registered_users_count.merge(active_users_count_with_cohorts, on = 'registration_date', how = 'left')

In [50]:
retention_table.head()

,registration_date,registered_users_count,activity_date,active_users_count
0,2019-05-01,6516,2019-05-01,2590
1,2019-05-01,6516,2019-05-02,2146
2,2019-05-01,6516,2019-05-03,1637
3,2019-05-01,6516,2019-05-04,1451
4,2019-05-01,6516,2019-05-05,1296


In [62]:
# Рассчитываем время между регистрацией и датой послденей активности пользователя, приводим к целочисленному значению
retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']
retention_table['lifetime'] = retention_table['lifetime'] / np.timedelta64(1, 'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int)
retention_table.head()

,registration_date,registered_users_count,activity_date,active_users_count,lifetime
0,2019-05-01,6516,2019-05-01,2590,0
1,2019-05-01,6516,2019-05-02,2146,1
2,2019-05-01,6516,2019-05-03,1637,2
3,2019-05-01,6516,2019-05-04,1451,3
4,2019-05-01,6516,2019-05-05,1296,4


In [52]:
# Получаем данные о выручке, приводим дату в нужный формат
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())
revenue = revenue.rename(columns = {'date':'activity_date'})
for col in ['activity_date', 'registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])

In [53]:
# Соединяем таблицу по выручке с таблицей о пользователях 
retention_table_with_revenue = retention_table.merge(revenue, on = ['registration_date', 'activity_date'], how = 'left')
retention_table_with_revenue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1891 entries, 0 to 1890
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   registration_date         1891 non-null   datetime64[ns]
 1   registered_users_count    1891 non-null   int64         
 2   activity_date             1891 non-null   datetime64[ns]
 3   active_users_count        1891 non-null   int64         
 4   lifetime                  1891 non-null   int64         
 5   users_count_with_revenue  1745 non-null   float64       
 6   revenue                   1745 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(3)
memory usage: 118.2 KB


In [54]:
# Заполняем пропуски нулями и приводим к целочисленному формату количество пользователей с выручкой
for col in ['users_count_with_revenue', 'revenue']:
    retention_table_with_revenue[col] = retention_table_with_revenue[col].fillna(0)
retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)
retention_table_with_revenue

,registration_date,registered_users_count,activity_date,active_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,6516,2019-05-01,2590,0,1672,2357.58000
1,2019-05-01,6516,2019-05-02,2146,1,1420,514.67900
2,2019-05-01,6516,2019-05-03,1637,2,1048,390.95200
3,2019-05-01,6516,2019-05-04,1451,3,931,260.13600
4,2019-05-01,6516,2019-05-05,1296,4,835,233.09400
...,...,...,...,...,...,...,...
1886,2019-06-30,3301,2019-07-26,193,26,118,5.43194
1887,2019-06-30,3301,2019-07-27,186,27,0,0.00000
1888,2019-06-30,3301,2019-07-28,198,28,0,0.00000
1889,2019-06-30,3301,2019-07-29,144,29,100,10.05400


In [55]:
# Создаем таблицу для расчета коэф-та удержания 7-го дня для МАЯ
retention_7_table_may = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>='2019-05-01'][retention_table_with_revenue['registration_date']<'2019-06-01']
retention_7_table_may.reset_index()
retention_7_table_may.head()

<ipython-input-55-a07666d2c045>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_table_may = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>='2019-05-01'][retention_table_with_revenue['registration_date']<'2019-06-01']


,registration_date,registered_users_count,activity_date,active_users_count,lifetime,users_count_with_revenue,revenue
7,2019-05-01,6516,2019-05-08,953,7,616,63.9065
38,2019-05-02,5309,2019-05-09,745,7,490,313.7220
69,2019-05-03,6266,2019-05-10,895,7,571,110.7260
100,2019-05-04,8639,2019-05-11,1186,7,750,124.8590
131,2019-05-05,9787,2019-05-12,1358,7,885,174.9240


In [56]:
# Рассчитываем коэф. удержания для МАЯ
retention_7_may = retention_7_table_may['active_users_count'].sum() / retention_7_table_may['registered_users_count'].sum()
retention_7_may

0.12933000957960866

#### Коэффициент удержания 7-го дня для мая составляет 12,93%.

In [57]:
# Создаем таблицу для расчета коэф-та удержания 7-го дня для ИЮНЯ
retention_7_table_june = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>='2019-06-01'][retention_table_with_revenue['registration_date']<'2019-07-01']
retention_7_table_june.head()

<ipython-input-57-0b67db4dd381>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_table_june = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>='2019-06-01'][retention_table_with_revenue['registration_date']<'2019-07-01']


,registration_date,registered_users_count,activity_date,active_users_count,lifetime,users_count_with_revenue,revenue
968,2019-06-01,4833,2019-06-08,545,7,365,176.3160
999,2019-06-02,5255,2019-06-09,683,7,461,193.8390
1030,2019-06-03,4193,2019-06-10,484,7,320,27.7548
1061,2019-06-04,4194,2019-06-11,511,7,320,54.9610
1092,2019-06-05,3998,2019-06-12,472,7,301,63.1554


In [58]:
# Рассчитываем коэф. удержания для ИЮНЯ
retention_7_june = retention_7_table_june['active_users_count'].sum() / retention_7_table_june['registered_users_count'].sum()
retention_7_june

0.12560867530346634

#### Коэффициент удержания 7-го дня для июня составляет 12,56%. Относительно мая коэффициент практически не уменьшился.

In [59]:
# Рассчитываем среднюю выручку от активного (платящего) пользователя для МАЯ
may_arppu = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"].sum() / retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"].sum()
may_arppu

0.3061245856847124

#### Средняя выручка с платящего пользователя для мая составляет 0,31.

In [60]:
# Рассчитываем среднюю выручку от активного (платящего) пользователя для ИЮНЯ
june_arppu = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"].sum() / retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"].sum()
june_arppu

0.3931940099426141

#### Средняя выручка с платящего пользователя для июня составляет 0,39%. Относительно мая коэффициент вырос, что говорит об эффективности внедрения фичи.

In [61]:
(june_arppu - may_arppu)/may_arppu*100

28.442480065151422

## Выводы:
### 1. Коэф. удержания 7-го дня остался примерно прежним.
### 2. Средний доход с 1-го платящего польщователя увеличился. Темп прироста составил 28,44%.
### В целом, мы можем сказать об эффективности внедрения фичи. Однако увеличение средней выручки и неизменность коэф. текучести может говорить о том, что есть другие факторы, повлиявшие на такие результаты. Возможно, стоит попытаться получить данные, которые характеризуют именно использование ленты (нашей фичи).